In [4]:
import numpy as np
import tespy
from matplotlib import pyplot as plt

from tespy.networks import Network
from tespy.components import (Sink, Source, Valve, SimpleHeatExchanger, Compressor, CycleCloser)
from tespy.components import CombustionChamber # adiabatic ?
from tespy.components import (HeatExchanger, Condenser, Desuperheater)
from tespy.components import (Drum, Merge, Separator, Splitter)
from tespy.components import (Pump, Turbine)
from tespy.connections import Connection, Bus
from tespy.tools import ExergyAnalysis
from CoolProp.CoolProp import PropsSI as cpsi

In [5]:
# to do
# add components dampftrommel/wärmeübertrager, see example --> evaporator drum
# add entgaser (what component(s))
# impose conditions after compressors, economizers and WÜ
# turbine later

In [21]:
# FLUIDS
wf = 'Water'
fld_wf = {wf:1}

gas = 'Methane' # 30 bar and 15°
fld_gas = {'CH4':1}

air = 'Air' # 1 bar and 15°
fld_air = {'N2': 0.79, 'O2': 0.21}

# NETWORK
dampferzeuger = Network(T_unit = 'C', p_unit = 'bar', h_unit = 'kJ / kg', m_unit = 't / h')

# COMPONENTS
# Sources
src_water = Source('Water_source')
src_gas   = Source('Erdgas_source')
src_air   = Source('Luft_source')
# Sinks
snk_high  = Sink('High_sink')
snk_mid   = Sink('Mid_sink')
snk_low   = Sink('Low_sink')
snk_abgas = Sink('Abgas_sink')
# Splitter
splitter = Splitter('Splitter', num_out=3)
# Combustion Chamber
cmp_cc = CombustionChamber('Verbrennungskammer')
# Compressors/Pumps
comp_high = Compressor('High_compressor')
comp_mid  = Compressor('Mid_compressor')
comp_low  = Compressor('Low_compressor')
comp_air  = Compressor('Frischluft-Verdichter')
# Economizers
eco_high = HeatExchanger('High_economizer')
eco_mid  = HeatExchanger('Mid_economizer')
eco_low  = HeatExchanger('Low_economizer')
# Heat Exchangers
ev_high = HeatExchanger('High_evaporator')
ev_mid  = HeatExchanger('Mid_evaporator')
ev_low  = HeatExchanger('Low_evaporator')
# Drums
drum_high = Drum('High_drum')
drum_mid  = Drum('Mid_drum')
drum_low  = Drum('Low_drum')


# CONNECTIONS

# Gas : from source to combustion chamber, and then through heat exchangers to the sink
c10 = Connection(src_gas,   'out1', cmp_cc,    'in1', label = '10')
c11 = Connection(cmp_cc,    'out1', ev_high,   'in1', label = '11')
c12 = Connection(ev_high,   'out1', ev_mid,    'in1', label = '12')
c13 = Connection(ev_mid,    'out1', ev_low,    'in1', label = '13')
c14 = Connection(ev_low,    'out1', eco_high,  'in1', label = '14')
c15 = Connection(eco_high,  'out1', eco_mid,   'in1', label = '15')
c16 = Connection(eco_mid,   'out1', eco_low,   'in1', label = '16')
c17 = Connection(eco_low,   'out1', snk_abgas, 'in1', label = '17')
# Air : from source to combustion chamber
c20 = Connection(src_air,  'out1', comp_air, 'in1', label = '20')
c21 = Connection(comp_air, 'out1', cmp_cc,   'in2', label = '21')
# Water (working fluid) : from the source, through splitter, compressor, economizer, drum/evaporator and then into sink
c30 = Connection(src_water, 'out1', splitter, 'in1', label = '30')

c31 = Connection(splitter, 'out1', comp_high, 'in1', label = '31')
c41 = Connection(splitter, 'out2', comp_mid,  'in1', label = '41')
c51 = Connection(splitter, 'out3', comp_low,  'in1', label = '51')

c32 = Connection(comp_high, 'out1', eco_high, 'in2', label = '32')
c42 = Connection(comp_mid,  'out1', eco_mid,  'in2', label = '42')
c52 = Connection(comp_low,  'out1', eco_low,  'in2', label = '52')

c33 = Connection(eco_high, 'out2', drum_high,  'in1', label = '33')
c43 = Connection(eco_mid,  'out2', drum_mid,   'in1', label = '43')
c53 = Connection(eco_low,  'out2', drum_low,   'in1', label = '53')

c34 = Connection(drum_high, 'out1', ev_high,  'in2', label = '34')
c44 = Connection(drum_mid,  'out1', ev_mid,   'in2', label = '44')
c54 = Connection(drum_low,  'out1', ev_low,   'in2', label = '54')

c35 = Connection(ev_high, 'out2', drum_high,  'in2', label = '35')
c45 = Connection(ev_mid,  'out2', drum_mid,   'in2', label = '45')
c55 = Connection(ev_low,  'out2', drum_low,   'in2', label = '55')

c36 = Connection(drum_high, 'out2', snk_high, 'in1', label = '36')
c46 = Connection(drum_mid,  'out2', snk_mid,  'in1', label = '46')
c56 = Connection(drum_low,  'out2', snk_low,  'in1', label = '56')

dampferzeuger.add_conns(c10, c11, c12, c13, c14, c15, c16, c17, c20, c21, c30, c31, c32, c33, c34, c35, c36, c41, c42, c43, c44, c45, c46, c51, c52, c53, c54, c55, c56)


# PARAMETERS

#components
#splitter.set_attr(pr = 1)

cmp_cc.set_attr(lamb = 1.05) # Luftzahl

comp_high.set_attr(eta_s = 0.8) # isentropischer Wirkungsgrad
comp_mid.set_attr(eta_s = 0.8)  # isentropischer Wirkungsgrad
comp_low.set_attr(eta_s = 0.8)  # isentropischer Wirkungsgrad
comp_air.set_attr(eta_s = 0.8)  # isentropischer Wirkungsgrad

ev_high.set_attr(pr1 = 1, pr2 = 1) # pressure ratios
ev_mid.set_attr(pr1 = 1, pr2 = 1)  # pressure ratios
ev_low.set_attr(pr1 = 1, pr2 = 1)  # pressure ratios

eco_high.set_attr(pr1 = 1, pr2 = 1) # pressure ratios
eco_mid.set_attr(pr1 = 1, pr2 = 1)  # pressure ratios
eco_low.set_attr(pr1 = 1, pr2 = 1)  # pressure ratios

#connections
c30.set_attr(p = 1, T = 60, m = 1350, fluid = fld_wf)
c31.set_attr(m = 90)
c41.set_attr(m = 360)
#c51.set_attr(m = 900)
c32.set_attr(p = 41) # Adding 1 to the pressure because of future pressure losses
c42.set_attr(p = 15) # Adding 1 to the pressure because of future pressure losses
c52.set_attr(p = 5)  # Adding 1 to the pressure because of future pressure losses

T_sat_high = cpsi('T', 'Q', 0, 'P', 41*1e5, wf) - 273.15
c33.set_attr(T = T_sat_high)
T_sat_mid = cpsi('T', 'Q', 0, 'P', 15*1e5, wf) - 273.15
c43.set_attr(T = T_sat_mid)
T_sat_low = cpsi('T', 'Q', 0, 'P', 5*1e5, wf) - 273.15
c53.set_attr(T = T_sat_low)

flux_test = 100
c10.set_attr(p = 30, T = 15, m = flux_test, fluid = fld_gas)
c20.set_attr(p = 1 , T = 15, fluid = fld_air)


motor_comp_air = Bus('Motor-for-air-compression')
motor_comp_air.add_comps({'comp' : comp_air, 'char' : 0.975, 'base' : 'bus'})
motor_high = Bus('High_motor')
motor_high.add_comps({'comp' : comp_high, 'char' : 0.975, 'base' : 'bus'})
motor_mid = Bus('Mid_motor')
motor_mid.add_comps({'comp' : comp_mid, 'char' : 0.975, 'base' : 'bus'})
motor_low = Bus('Low_motor')
motor_low.add_comps({'comp' : comp_low, 'char' : 0.975, 'base' : 'bus'})


dampferzeuger.add_busses(motor_comp_air, motor_high, motor_mid, motor_low)

# SOLVE
dampferzeuger.solve(mode='design')
dampferzeuger.print_results()


Singularity in jacobian matrix, calculation aborted! Make sure your network does not have any linear dependencies in the parametrisation. Other reasons might be
-> given temperature with given pressure in two phase region, try setting enthalpy instead or provide accurate starting value for pressure.
-> given logarithmic temperature differences or kA-values for heat exchangers, 
-> support better starting values.
-> bad starting value for fuel mass flow of combustion chamber, provide small (near to zero, but not zero) starting value.



 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 1.41e+09   | 0 %        | NaN        | NaN        | NaN        | NaN        | NaN        
Total iterations: 1, Calculation time: 0.08 s, Iterations per second: 12.52

##### RESULTS (Bus: Motor-for-air-compression) #####
+-------+-------------------+-------------+--------------+
|       |   component value |   bus value |   efficiency |
|-------+-------------------+-------------+--------------|
| total |         0.000e+00 |   0.000e+00 |          nan |
+-------+-------------------+-------------+--------------+
##### RESULTS (Bus: High_motor) #####
+-------+-------------------+-------------+--------------+
|       |   component value |   bus value |   efficiency |
|-------+-------------------+-------------+--------------|
| total |         0.000e+00 |   0.000e+00 |          nan |
+--